In [1]:
import os
import pandas as pd
import numpy as np
import re
from sklearn.metrics import f1_score
from imblearn.metrics import geometric_mean_score
from scipy.stats import gaussian_kde
import scipy.stats as stats
from sklearn.linear_model import LogisticRegression # We will use sklearn here as it is required in the next algorithms
import statsmodels.api as sm
from sklearn.metrics import roc_auc_score
from sklearn import metrics
from imblearn.under_sampling import EditedNearestNeighbours
from imblearn.combine import SMOTEENN
#grid search for hyperparameter tuning
from sklearn.ensemble import AdaBoostClassifier
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import HalvingGridSearchCV
from sklearn.preprocessing import StandardScaler
standardscaler = StandardScaler()
# FE und RE model
from linearmodels import PanelOLS
from linearmodels import RandomEffects
from linearmodels import PanelOLS
from regtabletotext import prettify_result, prettify_result
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
dataset_list = ["olist_customers_dataset","olist_geolocation_dataset","olist_order_items_dataset","olist_order_payments_dataset","olist_orders_dataset","olist_products_dataset","product_category_name_translation","optimal_pricing_dp"]
for csv in dataset_list:
    locals()[csv] = pd.read_csv("C:/Users/akankshasaxena3/Downloads/olist/"+csv+".csv")

In [3]:
pricing_df = pd.read_csv("C:/Users/akankshasaxena3/Downloads/olist/post_pre_intervention_df.csv")

C:\Users\akankshasaxena3\AppData\Local\Temp\ipykernel_25296\2329613644.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  pricing_df = pd.read_csv("C:/Users/akankshasaxena3/Downloads/olist/post_pre_intervention_df.csv")


In [4]:
olist_orders_dataset.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [5]:
olist_orders_dataset['order_purchase_timestamp'] = pd.to_datetime(olist_orders_dataset['order_purchase_timestamp']).dt.normalize()


In [6]:
olist_order_items_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [7]:
olist_customers_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


## Pre-processing of SIDRA demographic dataset

In [8]:
demographic_dataset = ["sex_ratio","median_age","aging_index"]
for xlsx in demographic_dataset:
    locals()[xlsx] = pd.DataFrame(pd.read_excel("C:/Users/akankshasaxena3/Downloads/olist/"+xlsx+".xlsx")) 


The dataset for estimating consumer purchase probability from Olist (Olist & Sionek) contains only demographic location as consumer information, which is not sufficient to capture the purchase probability of consumer at a given price. Hence, for other demographic information on consumer I use the data from  SIDRA which is an acronym to "Sistema IBGE de Recuperação Automática" and it is the system where IBGE makes aggregate data from their researches available. I have used three demographic characteristics from SIDRA:
1. median age
2. sex ratio
3. aging index

In [9]:
aging_index.head()

,Nível,Cód.,cities,aging index
0,BR,1.0,Brasil,55.24
1,GR,1.0,Norte,27.60
2,GR,2.0,Nordeste,47.89
3,GR,3.0,Sudeste,67.81
4,GR,4.0,Sul,65.61


In [10]:
print('Aging index dataset---------------------------------------------->>>>>')
print('# of unique values for Cód.='+str(aging_index['Cód.'].nunique()))
print('# of unique values for Nivel='+str(aging_index['Nível'].nunique()))
print('# of unique values for cities='+str(aging_index['cities'].nunique()))
print('number of records in dataset ='+ str(aging_index.shape[0]))

print('median age dataset---------------------------------------------->>>>>')
print('# of unique values for Cód.='+str(median_age['Cód.'].nunique()))
print('# of unique values for Nivel='+str(median_age['Nível'].nunique()))
print('# of unique values for cities='+str(median_age['cities'].nunique()))
print('number of records in dataset ='+ str(median_age.shape[0]))

print('sex ratio dataset---------------------------------------------->>>>>')
print('# of unique values for Cód.='+str(sex_ratio['Cód.'].nunique()))
print('# of unique values for Nivel='+str(sex_ratio['Nível'].nunique()))
print('# of unique values for cities='+str(sex_ratio['cities'].nunique()))
print('number of records in dataset ='+ str(sex_ratio.shape[0]))


Aging index dataset---------------------------------------------->>>>>
# of unique values for Cód.=6245
# of unique values for Nivel=8
# of unique values for cities=6236
number of records in dataset =6432
median age dataset---------------------------------------------->>>>>
# of unique values for Cód.=6245
# of unique values for Nivel=8
# of unique values for cities=6236
number of records in dataset =6432
sex ratio dataset---------------------------------------------->>>>>
# of unique values for Cód.=6245
# of unique values for Nivel=8
# of unique values for cities=6236
number of records in dataset =6432


In [11]:
aging_index['Nível'].unique(),median_age['Nível'].unique(),sex_ratio['Nível'].unique()

(array(['BR', 'GR', 'UF', 'MU', 'IN', 'IM', 'CU',
        'Fonte: IBGE - Censo Demográfico'], dtype=object),
 array(['BR', 'GR', 'UF', 'MU', 'IN', 'IM', 'CU',
        'Fonte: IBGE - Censo Demográfico'], dtype=object),
 array(['BR', 'GR', 'UF', 'MU', 'IN', 'IM', 'CU',
        'Fonte: IBGE - Censo Demográfico'], dtype=object))

In [12]:
median_age.describe()

,Cód.,Median age
count,6.431000e+03,6431.000000
mean,2.939003e+06,35.241487
std,1.312919e+06,4.420474
min,1.000000e+00,15.000000
25%,2.301356e+06,33.000000
50%,3.121407e+06,35.000000
75%,4.107140e+06,38.000000
max,5.300108e+06,53.000000


In [13]:
sex_ratio.describe()

,Cód.,sex ratio
count,6.431000e+03,6431.000000
mean,2.939003e+06,99.790204
std,1.312919e+06,8.164929
min,1.000000e+00,82.890000
25%,2.301356e+06,95.870000
50%,3.121407e+06,99.140000
75%,4.107140e+06,102.750000
max,5.300108e+06,443.640000


In [14]:
different_combinations_1 = sex_ratio[~sex_ratio.set_index(['cities','Nível','Cód.']).index.isin(median_age.set_index(['cities','Nível','Cód.']).index)]
different_combinations_2 = median_age[~median_age.set_index(['cities','Nível','Cód.']).index.isin(sex_ratio.set_index(['cities','Nível','Cód.']).index)]

len(different_combinations_1),len(different_combinations_1)

(0, 0)

In [15]:
different_combinations_1 = aging_index[~aging_index.set_index(['cities','Nível','Cód.']).index.isin(median_age.set_index(['cities','Nível','Cód.']).index)]
different_combinations_2 = median_age[~median_age.set_index(['cities','Nível','Cód.']).index.isin(aging_index.set_index(['cities','Nível','Cód.']).index)]

len(different_combinations_1),len(different_combinations_2)

(0, 0)

The three datasets, median_age, sex_ratio, and aging_index, share a common structure with similar combinations of columns: 'cities', 'Nível', and 'Cód.'. Each unique combination of these columns in any of the datasets forms a record. 

'cities', 'Nível', and 'Cód.' are identifiers that uniquely define records in each dataset. Merging based on these columns consolidates information across these datasets.

In [16]:
#merging different demographic datasets for Brazil on common columns for further integration in consumer df
merged_1 = pd.merge(aging_index, median_age, on=['cities','Nível','Cód.'], how='inner')
final_merged_df = pd.merge(merged_1, sex_ratio, on=['cities','Nível','Cód.'], how='inner',indicator=True)

In [17]:
different_combinations_df1 = final_merged_df[~final_merged_df.set_index(['cities','Nível','Cód.']).index.isin(sex_ratio.set_index(['cities','Nível','Cód.']).index)]


In [18]:
len(different_combinations_df1)

0

In [19]:
final_merged_df.shape

(6432, 7)

In [20]:
print('# of unique values for Cód.='+str(final_merged_df['Cód.'].nunique()))
print('# of unique values for Nivel='+str(final_merged_df['Nível'].nunique()))
print('# of unique values for cities='+str(final_merged_df['cities'].nunique()))
print('number of records in dataset ='+ str(final_merged_df.shape[0]))

# of unique values for Cód.=6245
# of unique values for Nivel=8
# of unique values for cities=6236
number of records in dataset =6432


final_merged_df contains the information for all three demographics median_age, sex_ratio, and aging_index. It contains exactly same number of total records, and unique values of columns : 'cities', 'Nível', and 'Cód.' . Hence, there is no duplicates or missing data in final_merged_df

In [21]:
aging_index.shape,median_age.shape,aging_index.shape,final_merged_df.shape

((6432, 4), (6432, 4), (6432, 4), (6432, 7))

In [22]:
final_merged_df.head()# demographic df

,Nível,Cód.,cities,aging index,Median age,sex ratio,_merge
0,BR,1.0,Brasil,55.24,35.0,94.25,both
1,GR,1.0,Norte,27.60,29.0,99.67,both
2,GR,2.0,Nordeste,47.89,33.0,93.54,both
3,GR,3.0,Sudeste,67.81,37.0,92.90,both
4,GR,4.0,Sul,65.61,36.0,94.99,both


Below I have defined two functions for preprocesisng the "cities" column in final_merged_df dataset. 
1. extract_state function: it is used to add a new "state" column to final_merged_df whose values is equal to the state information extracted "from city column 
2.remove_bracket_content function: This function is use to further clean the cities column after extracting state information from the dataset. It removes the information in between the brackets frm values in "city" column
3. finally I also change the name of a city 'Brasil' to 'brasilia' to match the olist consumer dataset

In [23]:
final_merged_df['cities'] = final_merged_df['cities'].astype("string")


In [24]:
final_merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6432 entries, 0 to 6431
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   Nível        6432 non-null   object  
 1   Cód.         6431 non-null   float64 
 2   cities       6431 non-null   string  
 3   aging index  6431 non-null   float64 
 4   Median age   6431 non-null   float64 
 5   sex ratio    6431 non-null   float64 
 6   _merge       6432 non-null   category
dtypes: category(1), float64(4), object(1), string(1)
memory usage: 358.2+ KB


Below I have defined two functions that 

In [25]:

def extract_state(city):
    if isinstance(city, str):
        # Pattern to match strings within parentheses or after a slash
        pattern = r'\((.*?)\)|\/(.*?)$'
        match = re.search(pattern, city)
        if match:
            # Check which group has a match and return it
            return match.group(1) if match.group(1) else match.group(2)
    return None


In [26]:
final_merged_df['state'] = final_merged_df['cities'].apply(extract_state)# preparing demographic df

In [27]:
final_merged_df['state']

0       None
1       None
2       None
3       None
4       None
        ... 
6427    None
6428      GO
6429    None
6430      DF
6431    None
Name: state, Length: 6432, dtype: object

In [28]:
final_merged_df[final_merged_df['state'].notnull()]['state']

33                                RO
34                                RO
35                                RO
36                                RO
37                                RO
                    ...             
6420    Paraguai - Ponta Porã/Brasil
6422                              MT
6426                              GO
6428                              GO
6430                              DF
Name: state, Length: 5673, dtype: object

In [29]:
def remove_bracket_content(city):
    # Pattern to match content within parentheses and content after a slash
    pattern = r'\s*\([^)]*\)|\/.*$'
    return re.sub(pattern, '', city)


In [30]:
final_merged_df['cities'] = final_merged_df['cities'].astype(str)


In [31]:
final_merged_df['cities'] = final_merged_df['cities'].apply(remove_bracket_content)


In [32]:
final_merged_df['cities']

0          Brasil
1           Norte
2        Nordeste
3         Sudeste
4             Sul
          ...    
6427      Formosa
6428      Goiânia
6429    Rio Verde
6430     Brasília
6431         <NA>
Name: cities, Length: 6432, dtype: object

In [33]:
final_merged_df['cities'] = final_merged_df['cities'].replace('Brasil', 'brasilia')

In [34]:
final_merged_df['cities'] = final_merged_df['cities'].str.lower()

In [35]:
final_merged_df

,Nível,Cód.,cities,aging index,Median age,sex ratio,_merge,state
0,BR,1.0,brasilia,55.24,35.0,94.25,both,None
1,GR,1.0,norte,27.60,29.0,99.67,both,None
2,GR,2.0,nordeste,47.89,33.0,93.54,both,None
3,GR,3.0,sudeste,67.81,37.0,92.90,both,None
4,GR,4.0,sul,65.61,36.0,94.99,both,None
...,...,...,...,...,...,...,...,...
6427,CU,5208004.0,formosa,34.93,32.0,97.64,both,None
6428,CU,5208707.0,goiânia,45.24,34.0,92.95,both,GO
6429,CU,5218805.0,rio verde,33.34,32.0,100.45,both,None
6430,CU,5300108.0,brasília,38.96,33.0,92.53,both,DF


In [36]:
final_merged_df['state'].isnull().sum()

759

In [37]:
final_merged_df = final_merged_df.dropna(subset=['state'])


In [38]:
final_merged_df.shape

(5673, 8)

In [39]:
final_merged_df.isnull().sum()

Nível          0
Cód.           0
cities         0
aging index    0
Median age     0
sex ratio      0
_merge         0
state          0
dtype: int64

In [40]:
print('# of unique values for Cód.='+str(final_merged_df['Cód.'].nunique()))
print('# of unique values for Nivel='+str(final_merged_df['Nível'].nunique()))
print('# of unique values for cities='+str(final_merged_df['cities'].nunique()))
print('# of unique values for cities-state combo='+str(final_merged_df[['cities','state']].drop_duplicates().shape))
print(final_merged_df[['cities','state','Cód.']].drop_duplicates().shape)

# of unique values for Cód.=5570
# of unique values for Nivel=2
# of unique values for cities=5316
# of unique values for cities-state combo=(5590, 2)
(5590, 3)


In [41]:
print('olist_customers_dataset----------------------------------->>>>>')
print('# of unique values for cities='+str(olist_customers_dataset['customer_city'].nunique()))
print('# of unique values for state='+str(olist_customers_dataset['customer_state'].nunique()))
print('# of unique values for cities-state combo='+str(olist_customers_dataset[['customer_city', 'customer_state']].drop_duplicates().shape))

olist_customers_dataset----------------------------------->>>>>
# of unique values for cities=4119
# of unique values for state=27
# of unique values for cities-state combo=(4310, 2)


In [42]:
different_combinations_1 = olist_customers_dataset[olist_customers_dataset.set_index(['customer_city','customer_state']).index.isin(final_merged_df.set_index(['cities','state']).index)]


In [43]:
olist_customers_dataset.shape

(99441, 5)

In [44]:
different_combinations_1.shape
#52538 records in olist consumer dataset have corresponding values in SIDRA demographic dataset
#since we only have city and state columns in olist dataset hence this combination is used from
#SIDRA to fill aging index, median age and sex ratio columns in olist dataset. 

(52538, 5)

In [45]:
final_merged_df[['cities','state','Cód.','Nível']].drop_duplicates().shape

(5673, 4)

In [46]:
duplicate_records = final_merged_df.groupby(['cities','state','Cód.']).size().reset_index(name='number_of_records')

In [47]:
final_merged_df[final_merged_df['cities'] == 'aracaju']

,Nível,Cód.,cities,aging index,Median age,sex ratio,_merge,state
1787,MU,2800308.0,aracaju,55.15,35.0,84.81,both,SE
6287,CU,2800308.0,aracaju,42.37,33.0,87.66,both,SE


In [48]:
final_merged_df['records_per_city'] = final_merged_df.groupby(['cities', 'state'])['Nível'].transform('count')

C:\Users\akankshasaxena3\AppData\Local\Temp\ipykernel_25296\199270470.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merged_df['records_per_city'] = final_merged_df.groupby(['cities', 'state'])['Nível'].transform('count')


In [49]:
final_merged_df[final_merged_df['records_per_city']>1]

,Nível,Cód.,cities,aging index,Median age,sex ratio,_merge,state,records_per_city
49,MU,1100205.0,porto velho,29.54,31.0,96.76,both,RO,2
202,MU,1501402.0,belém,59.49,36.0,88.18,both,PA,2
336,MU,1600303.0,macapá,21.92,28.0,95.66,both,AP,2
572,MU,2105302.0,imperatriz,35.28,31.0,92.78,both,MA,2
668,MU,2111300.0,são luís,43.88,33.0,87.23,both,MA,2
...,...,...,...,...,...,...,...,...,...
6412,CU,4316907.0,santa maria,82.48,37.0,90.43,both,RS,2
6422,CU,5103403.0,cuiabá,38.51,33.0,95.19,both,MT,2
6426,CU,5205109.0,catalão,51.51,35.0,99.91,both,GO,2
6428,CU,5208707.0,goiânia,45.24,34.0,92.95,both,GO,2


In [50]:
city_state_counts = final_merged_df.groupby(['cities']).size().reset_index(name='count')

In [51]:
city_state_counts[city_state_counts['count'] >1]

,cities,count
57,alagoinha,2
106,alto alegre,3
122,alto paraíso,2
137,alvorada,2
162,amparo,2
...,...,...
5251,várzea,2
5257,várzea grande,2
5264,wenceslau braz,2
5283,água boa,2


In [52]:
city_state_counts_2 = final_merged_df.groupby(['cities', 'state']).size().reset_index(name='count')
city_state_counts_2[city_state_counts_2['count'] >1]

,cities,state,count
257,aracaju,SE,2
298,araraquara,SP,2
305,araruama,RJ,2
386,atibaia,SP,2
419,bagé,RS,2
...,...,...,...
5187,teresina,PI,2
5233,toledo,PR,2
5342,ubá,MG,2
5356,umuarama,PR,2


In [53]:
city_state_counts = olist_customers_dataset.groupby(['customer_city', 'customer_state']).size().reset_index(name='count')
city_state_counts[city_state_counts['count'] >1].drop_duplicates()

,customer_city,customer_state,count
0,abadia dos dourados,MG,3
2,abaete,MG,12
3,abaetetuba,PA,11
4,abaiara,CE,2
5,abaira,BA,2
...,...,...,...
4305,xinguara,PA,9
4306,xique-xique,BA,3
4307,zacarias,SP,2
4308,ze doca,MA,5


In [54]:
different_combinations_1 = city_state_counts[city_state_counts.set_index(['customer_city','customer_state']).index.isin(city_state_counts_2.set_index(['cities','state']).index)]
different_combinations_1

,customer_city,customer_state,count
0,abadia dos dourados,MG,3
3,abaetetuba,PA,11
4,abaiara,CE,2
8,abdon batista,SC,1
9,abelardo luz,SC,6
...,...,...,...
4302,xavantina,SC,2
4303,xaxim,SC,10
4305,xinguara,PA,9
4306,xique-xique,BA,3


in final_merged_df some cities-state combination have two records, one record belongs to MU and other belongs to CU value of Nivel column. Simce we need unique record per city-state combination, for such records I take the average of values across records. 

In [55]:

# Calculate the counts for each (city, state) combination
city_state_counts = final_merged_df.groupby(['cities', 'state']).size()

# Filter rows where there are more than one record for the (city, state) combination
rows_to_replace = city_state_counts[city_state_counts > 1].index

# Replace values in the 'age' column with the average for the specified condition
for city, state in rows_to_replace:
    mask = (final_merged_df['cities'] == city) & (final_merged_df['state'] == state)
    average_age = final_merged_df.loc[mask, 'aging index'].mean()
    average_median_age =final_merged_df.loc[mask, 'Median age'].mean()
    avergae_sex_ratio = final_merged_df.loc[mask, 'sex ratio'].mean()
    final_merged_df.loc[mask, 'aging index'] = average_age
    final_merged_df.loc[mask, 'Median age'] = average_median_age
    final_merged_df.loc[mask, 'sex ratio'] = avergae_sex_ratio

print(final_merged_df)


     Nível       Cód.                                 cities  aging index  \
33      MU  1100015.0                  alta floresta d'oeste       44.610   
34      MU  1100023.0                              ariquemes       34.610   
35      MU  1100031.0                                 cabixi       64.140   
36      MU  1100049.0                                 cacoal       43.500   
37      MU  1100056.0                             cerejeiras       47.010   
...    ...        ...                                    ...          ...   
6420    CU  5006606.0  internacional de pedro juan caballero       34.030   
6422    CU  5103403.0                                 cuiabá       40.150   
6426    CU  5205109.0                                catalão       49.730   
6428    CU  5208707.0                                goiânia       51.125   
6430    CU  5300108.0                               brasília       42.745   

      Median age  sex ratio _merge                         state  \
33     

In [56]:
olist_customers_dataset.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state'],
      dtype='object')

In [57]:
final_merged_df.rename(columns={'cities': 'customer_city'}, inplace=True)
final_merged_df.rename(columns={'state': 'customer_state'}, inplace=True)

C:\Users\akankshasaxena3\AppData\Local\Temp\ipykernel_25296\893224227.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merged_df.rename(columns={'cities': 'customer_city'}, inplace=True)
C:\Users\akankshasaxena3\AppData\Local\Temp\ipykernel_25296\893224227.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merged_df.rename(columns={'state': 'customer_state'}, inplace=True)


In [58]:
final_merged_df[['customer_city','customer_state','aging index','Median age','sex ratio']].drop_duplicates().shape,final_merged_df[['customer_city','customer_state','aging index','Median age','sex ratio']].shape

((5590, 5), (5673, 5))

In [59]:
final_merged_df.drop(['Nível', 'Cód.'], axis=1, inplace=True)

C:\Users\akankshasaxena3\AppData\Local\Temp\ipykernel_25296\3475307331.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_merged_df.drop(['Nível', 'Cód.'], axis=1, inplace=True)


In [60]:
final_merged_df.shape,final_merged_df.drop_duplicates().shape

((5673, 7), (5590, 7))

In [61]:
final_merged_df = final_merged_df.drop_duplicates()

In [62]:
series_age_index = final_merged_df.set_index(['customer_city','customer_state'])['aging index']
series_median_age = final_merged_df.set_index(['customer_city','customer_state'])['Median age']
series_sex_ratio = final_merged_df.set_index(['customer_city','customer_state'])['sex ratio']


series_age_index

customer_city                          customer_state              
alta floresta d'oeste                  RO                              44.61
ariquemes                              RO                              34.61
cabixi                                 RO                              64.14
cacoal                                 RO                              43.50
cerejeiras                             RO                              47.01
                                                                       ...  
taquara - parobé - igrejinha           RS                              58.33
tramandaí - osório                     RS                              78.14
internacional de uruguaiana            Brasil                          56.74
internacional de corumbá               Brasil                          36.67
internacional de pedro juan caballero  Paraguai - Ponta Porã/Brasil    34.03
Name: aging index, Length: 5590, dtype: float64

In [63]:
series_median_age

customer_city                          customer_state              
alta floresta d'oeste                  RO                              34.0
ariquemes                              RO                              32.0
cabixi                                 RO                              38.0
cacoal                                 RO                              33.0
cerejeiras                             RO                              34.0
                                                                       ... 
taquara - parobé - igrejinha           RS                              36.0
tramandaí - osório                     RS                              38.0
internacional de uruguaiana            Brasil                          35.0
internacional de corumbá               Brasil                          31.0
internacional de pedro juan caballero  Paraguai - Ponta Porã/Brasil    30.0
Name: Median age, Length: 5590, dtype: float64

In [64]:
series_sex_ratio

customer_city                          customer_state              
alta floresta d'oeste                  RO                              101.82
ariquemes                              RO                               98.25
cabixi                                 RO                              102.15
cacoal                                 RO                               95.93
cerejeiras                             RO                               98.03
                                                                        ...  
taquara - parobé - igrejinha           RS                               95.17
tramandaí - osório                     RS                               93.09
internacional de uruguaiana            Brasil                           92.90
internacional de corumbá               Brasil                           98.13
internacional de pedro juan caballero  Paraguai - Ponta Porã/Brasil     95.08
Name: sex ratio, Length: 5590, dtype: float64

In [65]:
#for existing city-state records in olist customers dataset following columns,
#aging_index,Median_age,sex_ratio are mapped from the series generated above from final_merge_df
olist_customers_dataset['aging_index'] = olist_customers_dataset.apply(lambda row: series_age_index.get((row['customer_city'], row['customer_state']), None), axis=1)
olist_customers_dataset['Median_age'] = olist_customers_dataset.apply(lambda row: series_median_age.get((row['customer_city'], row['customer_state']), None), axis=1)
olist_customers_dataset['sex_ratio'] = olist_customers_dataset.apply(lambda row: series_sex_ratio.get((row['customer_city'], row['customer_state']), None), axis=1)

In [66]:
olist_customers_dataset.isnull().sum()

customer_id                     0
customer_unique_id              0
customer_zip_code_prefix        0
customer_city                   0
customer_state                  0
aging_index                 46903
Median_age                  46903
sex_ratio                   46903
dtype: int64

In [67]:
final_merged_df[final_merged_df['customer_city'] == 'aracaju']

,customer_city,aging index,Median age,sex ratio,_merge,customer_state,records_per_city
1787,aracaju,48.76,34.0,86.235,both,SE,2


In [68]:
common_cities = set(olist_customers_dataset['customer_city'].unique()).intersection(set(final_merged_df['customer_city'].unique()))

In [69]:
common_state = set(olist_customers_dataset['customer_state'].unique()).intersection(set(final_merged_df['customer_state'].unique()))

In [70]:
final_merged_df[['customer_city','customer_state']].shape,final_merged_df[['customer_city','customer_state']].drop_duplicates().shape

((5590, 2), (5590, 2))

In [71]:
common_cities_states = set(olist_customers_dataset[['customer_city','customer_state']]).intersection(set(final_merged_df[['customer_city','customer_state']]))

In [72]:
len(common_state)

27

In [73]:
len(common_cities)

2233

In [74]:
len(common_cities)/olist_customers_dataset['customer_city'].nunique() 
#for 54% cities in consumer dataset we have demographic information available

0.5421218742413207

In [75]:
len(common_state)/olist_customers_dataset['customer_state'].nunique() 
#for all states in consumer dataset we have demographic information available

1.0

In [76]:
len(common_cities_states)

2

In [77]:
final_merged_df.to_csv("C:/Users/akankshasaxena3/Downloads/olist/demographic_brasil.csv")

In [78]:
state_grouped_aging_index = final_merged_df.groupby('customer_state')['aging index'].mean()
state_grouped_Median_age = final_merged_df.groupby('customer_state')['Median age'].mean()
state_grouped_sex_ratio = final_merged_df.groupby('customer_state')['sex ratio'].mean()

In [79]:
state_grouped_aging_index

customer_state
AC                                    20.792727
AL                                    38.762647
AM                                    17.520645
AP                                    16.820937
BA                                    58.436115
Brasil                                46.705000
Brasil - Ciudad del Este/Paraguai     43.780000
Brasil - Rivera/Uruguai               75.610000
CE                                    55.382310
DF                                    42.745000
ES                                    65.530000
GO                                    65.823679
MA                                    35.252742
MG                                    78.079244
MS                                    45.086329
MT                                    40.264539
PA                                    27.800035
PB                                    59.317803
PE                                    47.680027
PE - Juazeiro/BA                      29.100000
PI                       

In [80]:
olist_customers_dataset.columns

Index(['customer_id', 'customer_unique_id', 'customer_zip_code_prefix',
       'customer_city', 'customer_state', 'aging_index', 'Median_age',
       'sex_ratio'],
      dtype='object')

In [81]:
# null values in columns: aging_index,Median_age,sex_ratio, are filled with avergae across states
olist_customers_dataset['aging_index'] = olist_customers_dataset.apply(
        lambda row: state_grouped_aging_index[row['customer_state']] if pd.isna(row['aging_index']) else row['aging_index'], 
        axis=1
    )
olist_customers_dataset['Median_age'] = olist_customers_dataset.apply(
        lambda row: state_grouped_Median_age[row['customer_state']] if pd.isna(row['Median_age']) else row['Median_age'], 
        axis=1
    )
olist_customers_dataset['sex_ratio'] = olist_customers_dataset.apply(
        lambda row: state_grouped_sex_ratio[row['customer_state']] if pd.isna(row['sex_ratio']) else row['sex_ratio'], 
        axis=1
    )

In [82]:
olist_customers_dataset.isnull().sum()

customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
aging_index                 0
Median_age                  0
sex_ratio                   0
dtype: int64

In [83]:
olist_customers_dataset.shape

(99441, 8)

In [84]:
olist_customers_dataset.to_csv("C:/Users/akankshasaxena3/Downloads/olist/cleaned_consumer_df.csv")

## Integration of SIDRA data to Olist Data

### Preparation of Olist data for customer transaction information

In this section I use two datasets from Olist which are integrated to provide customer transaction information. These datasets are as follows:

1.olist_orders_dataset: contains order information, which order was placed by which consumer and at what time. Addtionally it contains information about order status also, which is essential in estimating consuemr purchase probabilty. However this dataset doesnt provide information on which order contains which products. This is where the second olist_order_items_dataset is needed.

2.olist_order_items_dataset: This dataset contains information about what products are sold in each order placed by customer. There is also additional information like price ,freight value and seller of each product but is not necessary for our analysis, so is dropped from further analysis.

In [85]:
olist_orders_dataset.shape,olist_order_items_dataset.shape,olist_order_items_dataset.drop_duplicates().shape

((99441, 8), (112650, 7), (112650, 7))

In [86]:
olist_orders_dataset.isnull().sum()

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [87]:
olist_orders_dataset.drop(['order_approved_at', 'order_delivered_carrier_date','order_delivered_customer_date'], axis=1, inplace=True)

In [88]:
olist_orders_dataset.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_estimated_delivery_date    0
dtype: int64

In [89]:
olist_orders_dataset.shape,olist_orders_dataset.drop_duplicates().shape

((99441, 5), (99441, 5))

In [90]:
olist_orders_dataset['order_id'].drop_duplicates().shape

(99441,)

In [91]:
olist_orders_dataset['customer_id'].drop_duplicates().shape

(99441,)

In [92]:
olist_orders_dataset[['customer_id','order_id']].drop_duplicates().shape

(99441, 2)

In [93]:
olist_orders_dataset['order_purchase_timestamp'].drop_duplicates().shape

(634,)

In [94]:
olist_order_items_dataset.isnull().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [95]:
olist_order_items_dataset.drop_duplicates().shape,olist_order_items_dataset.shape

((112650, 7), (112650, 7))

In [96]:
olist_order_items_dataset['order_id'].nunique(),olist_order_items_dataset['order_id'].drop_duplicates().shape

(98666, (98666,))

In [97]:
olist_order_items_dataset['product_id'].nunique()

32951

In [98]:
olist_order_items_dataset[['order_id','product_id']].drop_duplicates().shape

(102425, 2)

In [99]:
olist_order_items_dataset[['order_id','product_id','order_item_id']].drop_duplicates().shape

(112650, 3)

In [100]:
order_id_prod_id_combination = olist_order_items_dataset[['order_id','product_id']].drop_duplicates()

In [101]:
order_id_with_existing_product_id = olist_orders_dataset[olist_orders_dataset['order_id'].isin(order_id_prod_id_combination['order_id'])]['order_id'].unique()

In [102]:
len(order_id_with_existing_product_id)

98666

In [103]:
olist_orders_dataset_filtered_df = olist_orders_dataset[olist_orders_dataset['order_id'].isin(order_id_with_existing_product_id)]
#keeping only those consumer records for which product_id exists

In [104]:
olist_orders_dataset_filtered_df

,order_id,customer_id,order_status,order_purchase_timestamp,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13,2018-02-26 00:00:00
...,...,...,...,...,...
99436,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2017-03-09,2017-03-28 00:00:00
99437,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2018-02-06,2018-03-02 00:00:00
99438,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2017-08-27,2017-09-27 00:00:00
99439,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2018-01-08,2018-02-15 00:00:00


In [105]:
order_id_prod_id_combination

,order_id,product_id
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f
2,000229ec398224ef6ca0657da4fc703e,c777355d18b72b67abbeef9df44fd0fd
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089
...,...,...
112645,fffc94f6ce00a00581880bf54a75a037,4aa6014eceb682077f9dc4bffebc05b0
112646,fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828
112647,fffce4705a9662cd70adb13d4a31832d,72a30483855e2eafc67aee5dc2560482
112648,fffe18544ffabc95dfada21779c9644f,9c422a519119dcad7575db5af1ba540e


In [106]:
df_all_combi_consumer_prod_id = olist_orders_dataset_filtered_df.merge(order_id_prod_id_combination,on='order_id',how='outer')

In [107]:
df_all_combi_consumer_prod_id.shape

(102425, 6)

In [108]:
df_all_combi_consumer_prod_id.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_estimated_delivery_date    0
product_id                       0
dtype: int64

In [109]:
df_all_combi_consumer_prod_id['order_id'].shape,df_all_combi_consumer_prod_id['order_id'].drop_duplicates().shape

((102425,), (98666,))

In [110]:
df_all_combi_consumer_prod_id.shape

(102425, 6)

In [111]:
df_all_combi_consumer_prod_id.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_estimated_delivery_date    0
product_id                       0
dtype: int64

In [112]:
df_all_combi_consumer_prod_id['order_status'].value_counts()

delivered      100196
shipped          1127
canceled          465
invoiced          322
processing        306
unavailable         7
approved            2
Name: order_status, dtype: int64

In [113]:
df_all_combi_consumer_prod_id = df_all_combi_consumer_prod_id.sort_values("order_purchase_timestamp")

In [114]:
df_all_combi_consumer_prod_id['purchase_proba'] = df_all_combi_consumer_prod_id['order_status'].apply(lambda x: 0 if x in ['canceled', 'unavailable'] else 1)

In [115]:
df_all_combi_consumer_prod_id.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_estimated_delivery_date    0
product_id                       0
purchase_proba                   0
dtype: int64

In [116]:
df_all_combi_consumer_prod_id['purchase_proba'].value_counts()
#imbalanced dataset

1    101953
0       472
Name: purchase_proba, dtype: int64

In [117]:
olist_customers_dataset.shape

(99441, 8)

In [118]:
olist_customers_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               99441 non-null  object 
 1   customer_unique_id        99441 non-null  object 
 2   customer_zip_code_prefix  99441 non-null  int64  
 3   customer_city             99441 non-null  object 
 4   customer_state            99441 non-null  object 
 5   aging_index               99441 non-null  float64
 6   Median_age                99441 non-null  float64
 7   sex_ratio                 99441 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 6.1+ MB


In [119]:
olist_customers_dataset['customer_id'].nunique()

99441

In [120]:
df_all_combi_consumer_prod_id['customer_id'].nunique()

98666

In [121]:
common_consumer_id = set(olist_customers_dataset['customer_id'].unique()).intersection(set(df_all_combi_consumer_prod_id['customer_id'].unique()))

In [122]:
len(common_consumer_id)

98666

### Integration of SIDRA With Olist customer transaction dataset

df_all_combi_consumer_prod_id: contains customer transaction information. It means that it shows what product is purchased by a customer and on which day. 

Next step is to integrate the SIDRA demograhic information for each consumer in this dataset. The SIDRA demographic information is available in olist_customers_dataset and hence this dataset is integrated with df_all_combi_consumer_prod_id.

In [123]:
final_consumer_df_2 = df_all_combi_consumer_prod_id.merge(olist_customers_dataset,on = 'customer_id',how = 'inner').drop_duplicates()

In [124]:
final_consumer_df_2['customer_id'].nunique()

98666

In [125]:
final_consumer_df_2.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_estimated_delivery_date    0
product_id                       0
purchase_proba                   0
customer_unique_id               0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
aging_index                      0
Median_age                       0
sex_ratio                        0
dtype: int64

In [126]:
final_consumer_df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102425 entries, 0 to 102424
Data columns (total 14 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       102425 non-null  object        
 1   customer_id                    102425 non-null  object        
 2   order_status                   102425 non-null  object        
 3   order_purchase_timestamp       102425 non-null  datetime64[ns]
 4   order_estimated_delivery_date  102425 non-null  object        
 5   product_id                     102425 non-null  object        
 6   purchase_proba                 102425 non-null  int64         
 7   customer_unique_id             102425 non-null  object        
 8   customer_zip_code_prefix       102425 non-null  int64         
 9   customer_city                  102425 non-null  object        
 10  customer_state                 102425 non-null  object        
 11  

### Integrating product information to the consumer transaction data

final_consumer_df_2 : This dataset now contains the customer transaction information(i.e products bought by customer) along with customer charactersitics. 

Next step is to add product related information to this data. Hence , pricing_df dataset is used. This dataset is obtained from Part 2-Dynamic Pricing Estimation model

In [127]:
pricing_df.drop(['Unnamed: 0'],axis=1, inplace=True)

In [128]:
pricing_df['order_purchase_timestamp'] = pd.to_datetime(pricing_df['order_purchase_timestamp'])


In [129]:
pricing_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99607 entries, 0 to 99606
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   product_id                99607 non-null  object        
 1   order_purchase_timestamp  99607 non-null  datetime64[ns]
 2   demand                    99607 non-null  int64         
 3   price                     99607 non-null  float64       
 4   freight_value             99607 non-null  float64       
 5   category_frequency        99607 non-null  float64       
 6   cumulative_demand         99607 non-null  float64       
 7   revenue                   99607 non-null  object        
 8   weekday_number            99607 non-null  int64         
 9   month_number              99607 non-null  int64         
 10  year_number               99607 non-null  int64         
 11  is_holiday                99607 non-null  bool          
 12  sales_margin      

In [130]:
common_product_id = set(final_consumer_df_2['product_id'].unique()).intersection(set(pricing_df['product_id'].unique()))

In [131]:
len(common_product_id),final_consumer_df_2['product_id'].nunique(),pricing_df['product_id'].nunique()

(32934, 32951, 32934)

In [132]:
consumer_df_final = final_consumer_df_2.merge(pricing_df,on = ['product_id','order_purchase_timestamp'],how = 'inner').drop_duplicates()

In [133]:
consumer_df_final.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_estimated_delivery_date    0
product_id                       0
purchase_proba                   0
customer_unique_id               0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
aging_index                      0
Median_age                       0
sex_ratio                        0
demand                           0
price                            0
freight_value                    0
category_frequency               0
cumulative_demand                0
revenue                          0
weekday_number                   0
month_number                     0
year_number                      0
is_holiday                       0
sales_margin                     0
log_price                        0
coef_log_price                   0
avg_price_last_3_days            0
category_mean_demand

In [134]:
consumer_df_final['product_id'].nunique()

32934

In [135]:
consumer_df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 116502 entries, 0 to 116501
Data columns (total 32 columns):
 #   Column                         Non-Null Count   Dtype         
---  ------                         --------------   -----         
 0   order_id                       116502 non-null  object        
 1   customer_id                    116502 non-null  object        
 2   order_status                   116502 non-null  object        
 3   order_purchase_timestamp       116502 non-null  datetime64[ns]
 4   order_estimated_delivery_date  116502 non-null  object        
 5   product_id                     116502 non-null  object        
 6   purchase_proba                 116502 non-null  int64         
 7   customer_unique_id             116502 non-null  object        
 8   customer_zip_code_prefix       116502 non-null  int64         
 9   customer_city                  116502 non-null  object        
 10  customer_state                 116502 non-null  object        
 11  

In [136]:
consumer_df_final.product_id.nunique()

32934

In [137]:
final_consumer_df_2.isnull().sum()

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_estimated_delivery_date    0
product_id                       0
purchase_proba                   0
customer_unique_id               0
customer_zip_code_prefix         0
customer_city                    0
customer_state                   0
aging_index                      0
Median_age                       0
sex_ratio                        0
dtype: int64

In [138]:
consumer_df_final.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_estimated_delivery_date', 'product_id', 'purchase_proba',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'aging_index', 'Median_age', 'sex_ratio', 'demand',
       'price', 'freight_value', 'category_frequency', 'cumulative_demand',
       'revenue', 'weekday_number', 'month_number', 'year_number',
       'is_holiday', 'sales_margin', 'log_price', 'coef_log_price',
       'avg_price_last_3_days', 'category_mean_demand',
       'product_category_name', 'optimal_prices_RF', 'post'],
      dtype='object')

In [139]:
consumer_df_final.to_csv("C:/Users/akankshasaxena3/Downloads/olist/consumer_df_final.csv")